In [1]:
import qsharp
import math

n = 5 # amount of qubits
m = 1 # number of marked items
optimalIterations = (math.pi / 4) * math.sqrt(math.pow(2, n)/m)
optimalIterations = math.floor(optimalIterations)

print("Optimal number of iterations for Grover's algorithm with {} qubits and {} marked items: {}".format(n, m, optimalIterations))

Optimal number of iterations for Grover's algorithm with 5 qubits and 1 marked items: 4


In [2]:
%%qsharp
import Microsoft.Quantum.Convert.*;
import Microsoft.Quantum.Math.*;
import Microsoft.Quantum.Arrays.*;
import Microsoft.Quantum.Measurement.*;
import Microsoft.Quantum.Diagnostics.*;

namespace Grover {

    operation Run(nQubits: Int, iterations: Int) : Result[] {
        // Use Grover's algorithm to find a particular marked state.
        let results = GroverSearch(nQubits, iterations, ReflectAboutMarked);
        return results;
    }

    operation GroverSearch(
        nQubits : Int,
        iterations : Int,
        phaseOracle : Qubit[] => Unit) : Result[] {

        use qubits = Qubit[nQubits];

        PrepareUniform(qubits);

        for _ in 1..iterations {
            phaseOracle(qubits);
            ReflectAboutUniform(qubits);
        }

        ApplyToEach(X, qubits);

        // Measure and return the answer.
        return MResetEachZ(qubits);
    }

    operation Oracle(inputQubits : Qubit[]): Unit is Adj + Ctl {
        let temp = [true, true, true, false, true];
        for i in 0..Length(temp)-1 {
            if temp[i] {
                X(inputQubits[i]);
            }
        }
    }

    operation ReflectAboutMarked(inputQubits : Qubit[]) : Unit {
        use outputQubit = Qubit();
        within {
            // We initialize the outputQubit to (|0⟩ - |1⟩) / √2, so that
            // toggling it results in a (-1) phase.
            X(outputQubit);
            H(outputQubit);
            // Flip the outputQubit for marked states.
            // Here, we get the state with alternating 0s and 1s by using the X
            // operation on every other qubit.
            
            Oracle(inputQubits);
        } apply {
            Controlled X(inputQubits, outputQubit);
        }
    }

    operation PrepareUniform(inputQubits : Qubit[]) : Unit is Adj + Ctl {
        for q in inputQubits {
            H(q);
        }
    }

    operation ReflectAboutAllOnes(inputQubits : Qubit[]) : Unit {
        Controlled Z(Most(inputQubits), Tail(inputQubits));
    }

    operation ReflectAboutUniform(inputQubits : Qubit[]) : Unit {
        within {
            // Transform the uniform superposition to all-zero.
            Adjoint PrepareUniform(inputQubits);
            // Transform the all-zero state to all-ones
            for q in inputQubits {
                X(q);
            }
        } apply {
            // Now that we've transformed the uniform superposition to the
            // all-ones state, reflect about the all-ones state, then let the
            // within/apply block transform us back.
            ReflectAboutAllOnes(inputQubits);
        }
    }
}

In [3]:
results = qsharp.run(f"Grover.Run({n}, {optimalIterations})", shots=1)

resultstr = ""
for r in results[0]:
    resultstr += "0" if r == 0 else "1"

print("Result: |{}⟩".format(resultstr))

Result: |11101⟩
